## Project Final for ECE 556: AI for Radar and Remote Sensing
Paul Delgado

In [1]:
# Python packages
import csv
import datetime
from itertools import product
import json
import os
import pathlib
import requests
import sys
import zipfile

In [2]:
# Third-party packages
import mne_bids
import numpy as np
import openneuro
import PIL.Image
import tensorflow as tf

In [3]:
#import caffe
from scipy import optimize
import scipy.io as sio

#### I) Constants

In [4]:
# Data settings
results_dirpath_str = os.path.normpath(os.path.join(os.getcwd(), 'results'))
print(results_dirpath_str)

C:\Users\PaulDRP\source\repos\eeg1\results


In [5]:
subjects_list = ['S1', 'S2', 'S3']
rois_list = ['VC']
network_name_str = 'VGG19'

In [6]:
# Images in figure 2
image_type = 'natural'
image_label_list = ['Img0009', 'Img0002', 'Img0001', 'Img0005', 'Img0036', 'Img0045', 'Img0031', 'Img0043']
max_iteration = 200

In [7]:
# Data Directory
dat_dirpath_str = os.path.normpath(os.path.join(os.getcwd(), '..', 'DeepImageReconstruction'))
print(dat_dirpath_str)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction


In [8]:
decoded_features_dirpath_str = os.path.normpath(os.path.join(dat_dirpath_str, 'data', 'decodedfeatures'))
print(decoded_features_dirpath_str)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\decodedfeatures


In [9]:
img_mean_fp = os.path.normpath(os.path.join(dat_dirpath_str, 'data', 'ilsvrc_2012_mean.npy'))
print(img_mean_fp)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\ilsvrc_2012_mean.npy


In [10]:
# CNN model
model_fp = os.path.normpath(os.path.join(dat_dirpath_str, 'net', 'VGG_ILSVRC_19_layers', 'VGG_ILSVRC_19_layers.caffemodel'))
prototxt_fp = os.path.normpath(os.path.join(dat_dirpath_str, 'net', 'VGG_ILSVRC_19_layers', 'VGG_ILSVRC_19_layers.prototxt'))
channel_swap = (2, 1, 0)
print(model_fp)
print(prototxt_fp)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction\net\VGG_ILSVRC_19_layers\VGG_ILSVRC_19_layers.caffemodel
C:\Users\PaulDRP\source\repos\DeepImageReconstruction\net\VGG_ILSVRC_19_layers\VGG_ILSVRC_19_layers.prototxt


In [11]:
feat_std_fp = os.path.normpath(os.path.join(dat_dirpath_str, 'data', 'estimated_vgg19_cnn_feat_std.mat'))
print(feat_std_fp)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\estimated_vgg19_cnn_feat_std.mat


In [12]:
dataset = 'ds001506'
subject = '01'
session = 'perceptionNaturalImageTraining01'
run = 1
#task = 'trance'
#suffix = 'inplane'
datatype = 'anat'

In [13]:
bids_root_dirpath_str = os.path.normpath(os.path.join(os.getcwd(), '..', dataset))

#### II) Helper Functions

In [14]:
def clip_extreme_value(img, pct=1):
    '''clip the pixels with extreme values'''
    if pct < 0:
        pct = 0.

    if pct > 100:
        pct = 100.

    img = np.clip(img, np.percentile(img, pct/2.),
                  np.percentile(img, 100-pct/2.))
    return img

In [15]:
def estimate_cnn_feat_std(cnn_feat):
    feat_ndim = cnn_feat.ndim
    feat_size = cnn_feat.shape
    # for the case of fc layers
    if feat_ndim == 1 or (feat_ndim == 2 and feat_size[0] == 1) or (feat_ndim == 3 and feat_size[1] == 1 and feat_size[2] == 1):
        cnn_feat_std = np.std(cnn_feat)
    # for the case of conv layers
    elif feat_ndim == 3 and (feat_size[1] > 1 or feat_size[2] > 1):
        num_of_ch = feat_size[0]
        # std for each channel
        cnn_feat_std = np.zeros(num_of_ch, dtype='float32')
        for j in range(num_of_ch):
            feat_ch = cnn_feat[j, :, :]
            cnn_feat_std[j] = np.std(feat_ch)
        cnn_feat_std = np.mean(cnn_feat_std)  # std averaged across channels
    return cnn_feat_std

In [16]:
def L2_loss(feat, feat0, mask=1.):
    d = feat - feat0
    loss = (d*d*mask).sum()
    grad = 2 * d * mask
    return loss, grad

In [17]:
def switch_loss_fun(loss_type):
    if loss_type == 'l2':
        return L2_loss
    elif loss_type == 'l1':
        return L1_loss
    elif loss_type == 'inner':
        return inner_loss
    elif loss_type == 'gram':
        return gram_loss
    else:
        raise ValueError('unknown loss function type!')

In [18]:
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    
    new_shape = tf.cast(shape * scale, tf.int32)
    
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

In [19]:
def create_feature_masks(features, masks=None, channels=None):
    feature_masks = {}
    for layer in features.keys():
        if (masks is None or masks == {} or masks == [] or (layer not in masks.keys())) and (channels is None or channels == {} or channels == [] or (layer not in channels.keys())):  # use all features and all channels
            feature_masks[layer] = np.ones_like(features[layer])
        elif isinstance(masks, dict) and (layer in masks.keys()) and isinstance(masks[layer], np.ndarray) and masks[layer].ndim == 3 and masks[layer].shape[0] == features[layer].shape[0] and masks[layer].shape[1] == features[layer].shape[1] and masks[layer].shape[2] == features[layer].shape[2]:  # 3D mask
            feature_masks[layer] = masks[layer]
        # 1D feat and 1D mask
        elif isinstance(masks, dict) and (layer in masks.keys()) and isinstance(masks[layer], np.ndarray) and features[layer].ndim == 1 and masks[layer].ndim == 1 and masks[layer].shape[0] == features[layer].shape[0]:
            feature_masks[layer] = masks[layer]
        elif (masks is None or masks == {} or masks == [] or (layer not in masks.keys())) and isinstance(channels, dict) and (layer in channels.keys()) and isinstance(channels[layer], np.ndarray) and channels[layer].size > 0:  # select channels
            mask_2D = np.ones_like(features[layer][0])
            mask_3D = np.tile(mask_2D, [len(channels[layer]), 1, 1])
            feature_masks[layer] = np.zeros_like(features[layer])
            feature_masks[layer][channels[layer], :, :] = mask_3D
        # use 2D mask select features for all channels
        elif isinstance(masks, dict) and (layer in masks.keys()) and isinstance(masks[layer], np.ndarray) and masks[layer].ndim == 2 and (channels is None or channels == {} or channels == [] or (layer not in channels.keys())):
            mask_2D_0 = masks[layer]
            mask_size0 = mask_2D_0.shape
            mask_size = features[layer].shape[1:]
            if mask_size0[0] == mask_size[0] and mask_size0[1] == mask_size[1]:
                mask_2D = mask_2D_0
            else:
                mask_2D = np.ones(mask_size)
                n_dim1 = min(mask_size0[0], mask_size[0])
                n_dim2 = min(mask_size0[1], mask_size[1])
                idx0_dim1 = np.arange(n_dim1) + \
                    round((mask_size0[0] - n_dim1)/2)
                idx0_dim2 = np.arange(n_dim2) + \
                    round((mask_size0[1] - n_dim2)/2)
                idx_dim1 = np.arange(n_dim1) + round((mask_size[0] - n_dim1)/2)
                idx_dim2 = np.arange(n_dim2) + round((mask_size[1] - n_dim2)/2)
                mask_2D[idx_dim1, idx_dim2] = mask_2D_0[idx0_dim1, idx0_dim2]
            feature_masks[layer] = np.tile(
                mask_2D, [features[layer].shape[0], 1, 1])
        else:
            feature_masks[layer] = 0

    return feature_masks

In [52]:
def img_deprocess(img, img_mean=np.float32([104, 117, 123])):
    '''convert from Caffe's input image layout'''
    return np.dstack((img + np.reshape(img_mean, (3, 1, 1)))[::-1])

In [53]:
def obj_fun(img, net, features, feature_masks, layer_weight, loss_fun, save_intermediate, save_intermediate_every, save_intermediate_path, save_intermediate_ext, save_intermediate_postprocess, loss_list=[]):
    # reshape img
    img_size = net.blobs['data'].data.shape[-3:]
    img = img.reshape(img_size)

    # save intermediate image
    t = len(loss_list)
    if save_intermediate and (t % save_intermediate_every == 0):
        img_mean = net.transformer.mean['data']
        save_path = os.path.join(save_intermediate_path, '%05d.%s' % (t, save_intermediate_ext))
        if save_intermediate_postprocess is None:
            snapshot_img = img_deprocess(img, img_mean)
        else:
            snapshot_img = save_intermediate_postprocess(img_deprocess(img, img_mean))
        PIL.Image.fromarray(snapshot_img).save(save_path)

    # layer_list
    layer_list = features.keys()
    layer_list = sort_layer_list(net, layer_list)

    # num_of_layer
    num_of_layer = len(layer_list)

    # cnn forward
    net.blobs['data'].data[0] = img.copy()
    net.forward(end=layer_list[-1])

    # cnn backward
    loss = 0.
    layer_start = layer_list[-1]
    net.blobs[layer_start].diff.fill(0.)
    for j in xrange(num_of_layer):
        layer_start_index = num_of_layer - 1 - j
        layer_end_index = num_of_layer - 1 - j - 1
        layer_start = layer_list[layer_start_index]
        if layer_end_index >= 0:
            layer_end = layer_list[layer_end_index]
        else:
            layer_end = 'data'
        feat_j = net.blobs[layer_start].data[0].copy()
        feat0_j = features[layer_start]
        mask_j = feature_masks[layer_start]
        layer_weight_j = layer_weight[layer_start]
        loss_j, grad_j = loss_fun(feat_j, feat0_j, mask_j)
        loss_j = layer_weight_j * loss_j
        grad_j = layer_weight_j * grad_j
        loss = loss + loss_j
        g = net.blobs[layer_start].diff[0].copy()
        g = g + grad_j
        net.blobs[layer_start].diff[0] = g.copy()
        if layer_end == 'data':
            net.backward(start=layer_start)
        else:
            net.backward(start=layer_start, end=layer_end)
        net.blobs[layer_start].diff.fill(0.)
    grad = net.blobs['data'].diff[0].copy()

    # reshape gradient
    grad = grad.flatten().astype(np.float64)
    loss_list.append(loss)
    return loss, grad

In [54]:
def reconstruct_image(features, net,
                      layer_weight=None, 
                      channel=None, 
                      mask=None, 
                      initial_image=None, 
                      loss_type='l2', 
                      maxiter=500, 
                      disp=True, 
                      save_intermediate=False, 
                      save_intermediate_every=1, 
                      save_intermediate_path=None,
                      save_intermediate_ext='jpg'):
    # loss function
    #loss_fun = switch_loss_fun(loss_type)
    loss_fun = tf.keras.losses.MeanSquaredError()

    # make dir for saving intermediate
    if save_intermediate:
        if save_intermediate_path is None:
            fn_temp1 = datetime.datetime.now().strftime('%Y%m%dT%H%M%S')
            fn_name = 'recon_img_lbfgs_snapshots' + fn_temp1
            fp_relative = os.path.join(os.getcwd(), fn_name)
            save_intermediate_path = os.path.normpath(fp_relative)
        if not os.path.exists(save_intermediate_path):
            os.makedirs(save_intermediate_path)

    # image size
    #img_size = net.blobs['data'].data.shape[-3:]
    img_size = net.layers[0].shape[-3:]
    print(f'image size: {img_size}')

    # num of pixel
    num_of_pix = np.prod(img_size)
    print(f'num of pix: {num_of_pix}')

    # image mean
    #img_mean = net.transformer.mean['data']
    img_mean_fp = os.path.normpath(os.path.join(dat_dirpath_str, 'data', 'ilsvrc_2012_mean.npy'))
    img_mean = np.load(img_mean_fp)
    print(type(img_mean))
    img_mean = np.float32([img_mean[0].mean(), img_mean[1].mean(), img_mean[2].mean()])
    print(f'image mean: {img_mean.shape}')

    # img bounds
    img_min = -img_mean
    print(f'image min: {img_min.shape}')
    
    img_max = img_min + 255.
    
    pix_temp = num_of_pix / 3
    
    img_min0_tuple = (img_min[0], img_max[0])
    img_min0_list = [img_min0_tuple]
    img_min0 = np.array(img_min0_list) * pix_temp
    
    img_min1_tuple = (img_min[1], img_max[1])
    img_min1_list = [img_min1_tuple]
    img_min1 = np.array(img_min1_list) * pix_temp
    
    img_min2_tuple = (img_min[2], img_max[2])
    img_min2_list = [img_min2_tuple]
    img_min2 = np.array(img_min2_list) * pix_temp
    
    img_bounds = img_min0 + img_min1 + img_min2
    
    # initial image
    if initial_image is None:
        initial_image = np.random.randint(0, 256, (img_size[1], img_size[2], img_size[0]))
    if save_intermediate:
        save_name = 'initial_img.png'
        fp = os.path.join(save_intermediate_path, save_name)
        PIL.Image.fromarray(np.uint8(initial_image)).save(fp)

    # preprocess initial img
    #initial_image = img_preprocess(initial_image, img_mean)
    initial_image = tf.keras.applications.vgg19.preprocess_input(initial_image * img_mean)
    initial_image = initial_image.flatten()

    # layer_list
    layer_list = features.keys()
    print(layer_list)
    #layer_list = sort_layer_list(net, layer_list) # PD: already in order

    # number of layers
    num_of_layer = len(layer_list)

    # layer weight
    if layer_weight is None:
        weights = np.ones(num_of_layer)
        weights = np.float32(weights)
        weights = weights / weights.sum()
        layer_weight = {}
        for j, layer in enumerate(layer_list):
            layer_weight[layer] = weights[j]

    # feature mask
    feature_masks = create_feature_masks(features, masks=mask, channels=channel)

    # optimization params
    loss_list = []
    opt_params = {
        'args': (net, features, feature_masks, layer_weight, loss_fun, save_intermediate, save_intermediate_every, save_intermediate_path, save_intermediate_ext, loss_list),
        'method': 'L-BFGS-B',
        'jac': True,
        'bounds': img_bounds,
        'options': {'maxiter': maxiter, 'disp': disp},
    }
    
    print(type(features))
    
    # optimization
    #res = optimize.minimize(obj_fun, initial_image, **opt_params)
    opt = tf.keras.optimizers.SGD()
    #opt_op = opt.minimize(loss_fun)
    #opt_op.run()

    # recon img
    #img = res.x
    #img = net(
    img = initial_image.reshape(img_size)

    # return img
    return img_deprocess(img, img_mean), loss_list

#### III) Load Input Files & Create Directories

In [22]:
img_mean = np.load(img_mean_fp)

In [23]:
model = tf.keras.applications.vgg19.VGG19(include_top=True)

In [24]:
# Feature SD estimated from true CNN features of 10000 images
feat_std0 = sio.loadmat(feat_std_fp)

In [25]:
if not os.path.isdir(bids_root_dirpath_str):
    os.makedirs(bids_root_dirpath_str)

In [26]:
if not os.path.exists(results_dirpath_str):
    os.makedirs(results_dirpath_str)

#### IV) Prepare Dataset

In [27]:
# Download Data (BIDS-standard repo)
#openneuro.download(dataset=dataset, target_dir=bids_root, include=[f'sub-{subject}'])

In [28]:
bids_path = mne_bids.BIDSPath(root=bids_root_dirpath_str, session=session, datatype=datatype)

#### V) Load Data

In [29]:
raw = mne_bids.read_raw_bids(bids_path=bids_path)

ValueError: Suffix anat is not allowed. Use one of these suffixes ['meg', 'markers', 'eeg', 'ieeg', 'T1w', 'FLASH', 'participants', 'scans', 'electrodes', 'optodes', 'channels', 'coordsystem', 'events', 'headshape', 'digitizer', 'beh', 'physio', 'stim', 'nirs'].

In [ ]:
print(raw.info['subject_info'])

In [ ]:
print(raw.info['sfreq'])

In [ ]:
print(raw.annotations)

In [ ]:
print(raw.annotations[0])

In [ ]:
print(raw.annotations[1])

In [ ]:
print(raw.annotations[2])

In [ ]:
raw.plot()

In [ ]:
mne_bids.inspect_dataset(bids_root_dirpath_str)

#### VI) Format BIDS into Tensorflow Dataset

In [ ]:
#dataset = tf.data.Dataset.from_tensor_slices([1, 2, 3])

#### VII) Initialize Given Variables

In [30]:
# Average image of ImageNet
img_mean_ndarray = np.float32([img_mean[0].mean(), img_mean[1].mean(), img_mean[2].mean()])
print(img_mean_ndarray)

[104.00699 116.66877 122.67892]


In [31]:
#net = caffe.Classifier(prototxt_file, model_file, mean=img_mean, channel_swap=channel_swap)
print(f'initial input shape: {model.input.shape}')
model.layers[0].shape = (10, 3, 224, 224) #in1 = tf.keras.layers.Reshape((10, 3, 224, 224))

#h, w = net.blobs['data'].data.shape[-2:]
h, w = model.input.shape[1:3]
print(f'height: {h} and width: {w}')

#net.blobs['data'].reshape(1, 3, h, w)
model.layers[0].shape = (1, 3, h, w)
print(f'model reshaped: {model.input.shape}')

initial input shape: (None, 224, 224, 3)
height: 224 and width: 224
model reshaped: (None, 224, 224, 3)


In [33]:
# Initial image for the optimization (here we use the mean of ilsvrc_2012_mean.npy as RGB values)
img_init_ndarray = np.zeros((h, w, 3), dtype='float32')
img_init_ndarray[:, :, 0] = img_mean_ndarray[2].copy()
img_init_ndarray[:, :, 1] = img_mean_ndarray[1].copy()
img_init_ndarray[:, :, 2] = img_mean_ndarray[0].copy()
print(f'initial image shape: {img_init_ndarray.shape}')

initial image shape: (224, 224, 3)


In [34]:
# CNN Layers (all conv and fc layers)
layers = []
count = 0
pool_count = 1
block_count = 1
for layer in model.layers:
    if layer.__class__.__name__ == 'InputLayer':
        print(f'layer {count}: {layer.name}')
    elif layer.__class__.__name__ == 'Conv2D':
        sections = layer.name.split('_')
        block = sections[0]
        blockNum = block[-1]
        if int(blockNum) > block_count:
            block_count = int(blockNum)
        layerNum = sections[1][-1]
        val = 'conv' + blockNum + '_' + layerNum
        print(f'layer {count}: {layer.name} is {val}')
        layers.append(val)
    elif layer.__class__.__name__ == 'MaxPooling2D':
        sections = layer.name.split('_')
        block = sections[0]
        blockNum = block[-1]
        if int(blockNum) > block_count:
            block_count = int(blockNum)
        val = 'pool' + blockNum
        print(f'layer {count}: {layer.name} is {val}')
        layers.append(val)
        pool_count = pool_count + 1
    elif layer.__class__.__name__ == 'Flatten':
        print(f'layer {count}: Flatten')
    elif layer.__class__.__name__ == 'Dense':
        block_count = block_count + 1
        val = 'fc' + str(block_count)
        print(f'layer {count}: {layer.name} is {val}')
        layers.append(val)
    else:
        print(f'Error finding layer {count}: {layer.name}')
    count = count + 1

layer 0: input_1
layer 1: block1_conv1 is conv1_1
layer 2: block1_conv2 is conv1_2
layer 3: block1_pool is pool1
layer 4: block2_conv1 is conv2_1
layer 5: block2_conv2 is conv2_2
layer 6: block2_pool is pool2
layer 7: block3_conv1 is conv3_1
layer 8: block3_conv2 is conv3_2
layer 9: block3_conv3 is conv3_3
layer 10: block3_conv4 is conv3_4
layer 11: block3_pool is pool3
layer 12: block4_conv1 is conv4_1
layer 13: block4_conv2 is conv4_2
layer 14: block4_conv3 is conv4_3
layer 15: block4_conv4 is conv4_4
layer 16: block4_pool is pool4
layer 17: block5_conv1 is conv5_1
layer 18: block5_conv2 is conv5_2
layer 19: block5_conv3 is conv5_3
layer 20: block5_conv4 is conv5_4
layer 21: block5_pool is pool5
layer 22: Flatten
layer 23: fc1 is fc6
layer 24: fc2 is fc7
layer 25: predictions is fc8


In [37]:
opts_dict = {
    'loss_type': 'l2', # The loss function type: {'l2','l1','inner','gram'}
    'maxiter': max_iteration, # The maximum number of iterations
    'initial_image': None, # (setting to None will use random noise as initial image)
    'disp': True # Display the information on the terminal or not
}
print(opts_dict)

{'loss_type': 'l2', 'maxiter': 200, 'initial_image': None, 'disp': True}


#### VII) Main Process

In [41]:
# Save the optional parameters
opts_json_str = json.dumps(opts_dict)
print(opts_json_str)
opts_fn = os.path.normpath(os.path.join(results_dirpath_str, 'options.json'))
options_file_handler = open(opts_fn, 'w')
options_file_handler.write(obj_json_str)
options_file_handler.close()

{"loss_type": "l2", "maxiter": 200, "initial_image": null, "disp": true}


In [55]:
# Reconstrucion
for subject, roi, image_label in product(subjects_list, rois_list, image_label_list):
    print(f'Subject:     {subject}')
    print(f'ROI:         {roi}')
    print(f'Image label: {image_label}')
    print('')

    save_dir = os.path.join(results_dirpath_str, subject, roi)
    print(f'save dir: {save_dir}')
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Load the decoded CNN features
    features = {}
    print('loading decoded CNN features')
    for layer in layers:
        # The file full name depends on the data structure for decoded CNN features
        name = f'{image_type}-{network_name_str}-{layer}-{subject}-{roi}-{image_label}.mat'
        file_name = os.path.join(decoded_features_dirpath_str, image_type, network_name_str, layer, subject, roi, name)
        feat = sio.loadmat(file_name)['feat']
        if 'fc' in layer:
            feat = feat.reshape(feat.size)

        # Correct the norm of the decoded CNN features
        feat_std = estimate_cnn_feat_std(feat)
        feat = (feat / feat_std) * feat_std0[layer]
        features.update({layer: feat})

    # Weight of each layer in the total loss function

    # Norm of the CNN features for each layer
    feat_norm = np.array([np.linalg.norm(features[layer]) for layer in layers], dtype='float32')

    # Use the inverse of the squared norm of the CNN features as the weight for each layer
    weights = 1. / (feat_norm ** 2)

    # Normalise the weights such that the sum of the weights = 1
    weights = weights / weights.sum()
    layer_weight = dict(zip(layers, weights))

    opts_dict.update({'layer_weight': layer_weight})

    # Reconstruction
    snapshots_dir = os.path.join(save_dir, 'snapshots', f'image-{image_label}')
    recon_img, loss_list = reconstruct_image(features,
                                             model,
                                             save_intermediate=True,
                                             save_intermediate_path=snapshots_dir,
                                             **opts_dict)

    # Save the results

    # Save the raw reconstructed image
    save_name = 'recon_img' + '-' + image_label + '.mat'
    sio.savemat(os.path.join(save_dir, save_name), {'recon_img': recon_img})

    # To better display the image, clip pixels with extreme values (0.02% of pixels with extreme low values
    # and 0.02% of the pixels with extreme high values). And then normalise the image by mapping
    # the pixel value to be within [0,255].
    save_name = 'recon_img_normalized' + '-' + image_label + '.jpg'
    save_fp = os.path.join(save_dir, save_name)
    recon_img = clip_extreme_value(recon_img, pct=4)
    img_np = tf.image.per_image_standardization(recon_img)
    PIL.Image.fromarray(img_np).save(save_fp)


Subject:     S1
ROI:         VC
Image label: Img0009

save dir: C:\Users\PaulDRP\source\repos\eeg1\results\S1\VC
loading decoded CNN features
image size: (3, 224, 224)
num of pix: 150528
<class 'numpy.ndarray'>
image mean: (3,)
image min: (3,)
dict_keys(['conv1_1', 'conv1_2', 'pool1', 'conv2_1', 'conv2_2', 'pool2', 'conv3_1', 'conv3_2', 'conv3_3', 'conv3_4', 'pool3', 'conv4_1', 'conv4_2', 'conv4_3', 'conv4_4', 'pool4', 'conv5_1', 'conv5_2', 'conv5_3', 'conv5_4', 'pool5', 'fc6', 'fc7', 'fc8'])
<class 'dict'>


AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute '__array_interface__'

### Build model

In [ ]:
model.compile()
#model.compile(loss=tf.keras.losses.MeanAbsoluteError(), optimizer=tf.keras.optimizers.Adam(), metrics=tf.keras.metrics.Accuracy())

In [ ]:
model.summary()

#### Train

In [ ]:
#model.fit(, epochs=10)

#### Evaluate

In [ ]:
#loss, accuracy = model.evaluate(, verbose=2)